In [1]:
%load_ext autoreload

%autoreload 2

In [2]:

import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import glob
import numpy as np
import statistics as stats
import scipy
import math
import matplotlib.pyplot as plt
import random
import h5py
from importlib import reload

from src.tf_tools.rnn_regression import coolRNN
from src.tf_tools.cnn_regression import originalResNet, tranferNet, MCDropout
from src.grammar.sentences import sentences_Cspacer

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from scipy.stats import linregress
from sklearn.metrics import ConfusionMatrixDisplay

import shap
from deeplift.visualization import viz_sequence
# 3.10.4

2024-03-18 12:04:27.137582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 12:04:27.137644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 12:04:27.139444: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 12:04:28.438621: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
folder = 'modisco/activity'

#Load in hypothetical importance scores
rawhyp_impscores = np.load(folder+'/sv.npy')
#Normalize scores across each basepair by making the mean contribution zero
hyp_impscores = [x - np.mean(x, axis=-1)[:,None] for x in rawhyp_impscores]
#hyp_impscores = rawhyp_impscores

#Load in one hot encoded seq
onehot_data = np.load(folder+'/ohe.npy')
# Multiply to get actual importance scores
impscores = [x*y for x,y in zip(hyp_impscores, onehot_data)]


nulldist_perposimp = np.sum(impscores[-100:], axis=-1)

((4, 164), (4, 164))

In [4]:
import h5py
import numpy as np
%matplotlib inline
import modisco
reload(modisco)
import modisco.backend
reload(modisco.backend.tensorflow_backend)
reload(modisco.backend)
import modisco.nearest_neighbors
reload(modisco.nearest_neighbors)
import modisco.affinitymat
reload(modisco.affinitymat.core)
reload(modisco.affinitymat.transformers)
import modisco.tfmodisco_workflow.seqlets_to_patterns
reload(modisco.tfmodisco_workflow.seqlets_to_patterns)
import modisco.tfmodisco_workflow.workflow
reload(modisco.tfmodisco_workflow.workflow)
import modisco.tfmodisco_workflow
reload(modisco.tfmodisco_workflow)
import modisco.aggregator
reload(modisco.aggregator)
import modisco.cluster
reload(modisco.cluster.core)
reload(modisco.cluster.phenograph.core)
reload(modisco.cluster.phenograph.cluster)
import modisco.value_provider
reload(modisco.value_provider)
import modisco.core
reload(modisco.core)
import modisco.coordproducers
reload(modisco.coordproducers)
import modisco.metaclusterers
reload(modisco.metaclusterers)
import modisco.clusterinit.memeinit
reload(modisco.clusterinit.memeinit)

Instructions for updating:
non-resource variables are not supported in the long term


TF-MoDISco is using the TensorFlow backend.
TF-MoDISco is using the TensorFlow backend.


<module 'modisco.clusterinit.memeinit' from '/ref/bclab/software/spack-0.17.2/var/spack/environments/tensorflow-gpu/.spack-env/._view/gctfpgbf2atvptgnwhwmsyrtv7diigjw/lib/python3.10/site-packages/modisco/clusterinit/memeinit.py'>

In [19]:

modisco_results = modisco.tfmodisco_workflow.workflow.TfModiscoWorkflow(
    target_seqlet_fdr=0.25,
    sliding_window_size=15,
    flank_size=5,
    max_seqlets_per_metacluster=50000,
    seqlets_to_patterns_factory=modisco.tfmodisco_workflow.seqlets_to_patterns.TfModiscoSeqletsToPatternsFactory(
        trim_to_window_size=15,initial_flank_to_add=5,final_min_cluster_size=30
    )
)(
    task_names=["task0"],
    contrib_scores={'task0': impscores},                
    hypothetical_contribs={'task0': hyp_impscores},
    one_hot=onehot_data,
    null_per_pos_scores={'task0': nulldist_perposimp}
)


MEMORY 5.362597888
On task task0
Computing windowed sums on original
Generating null dist
Computing threshold


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [ ]:
with h5py.File('data.h5', 'r') as file:
    print("Keys: %s" % file.keys())
    a_group_key = list(file.keys())[0]
     
    # Getting the data
    data = list(file[a_group_key])
    print(data)

: 